In [1]:
import pandas as pd

In [2]:
file = "190808_Panama_agosto_19.cohort.combined.hf.vcf"

In [3]:
def import_VCF42_cohort_pandas(vcf_file, sep='\t'):
    """
    Script to read vcf 4.2 cohort/join called vcf handling header lines
    """
    header_lines = 0
    with open(vcf_file) as f:
        first_line = f.readline().strip()
        next_line = f.readline().strip()
        while next_line.startswith("##"):
            header_lines = header_lines + 1
            #print(next_line)
            next_line = f.readline()
    
    if first_line.endswith('VCFv4.2'):
        dataframe = pd.read_csv(vcf_file, sep=sep, skiprows=[header_lines], header=header_lines)
    else:
        print("This vcf file is not v4.2")
        sys.exit(1)
           
    return dataframe

In [184]:
def repare_nongenotyped(vcf_file, nocall_fr=0.1):
    
    with open(vcf_file) as f:
        header = ""
        next_line = f.readline().strip()
        while next_line.startswith("##"):
            header = header + next_line
            #print(next_line)
            next_line = f.readline()
            
    df = import_VCF42_cohort_pandas(vcf_file)
    sample_list = df.columns[9:].tolist()
    #remove positions which haven't been enotyped in 0.2% or more samples
    for index, data_row in df.iloc[:,9:].iterrows():
        if any(x.startswith("./.") for x in data_row):
            non_genotyped = [x.startswith("./.") for x in data_row] #True False array
            non_genotyped_count = sum(non_genotyped) #True = 1, False = 0
            #Drop positions
            if non_genotyped_count / len(non_genotyped) > nocall_fr:
                #rint(index, df.loc[index, 'POS'])
                df.drop(index, inplace=True)
    #Reset index with new dataframe without highly non genotyped positions                                
    df = df.reset_index(drop=True)
        
    for index, data_row in df.iloc[:,9:].iterrows():
        if any(x.startswith("./.") for x in data_row):
            for sample in sample_list:
                sample_format = df.loc[index,sample]
                if sample_format.startswith("./."):
                    s_format_split = sample_format.split(":")
                    #FORMAT = GT:AD:DP:GQ:PGT:PID:PL:PS
                    if len(s_format_split) > 2:
                        gt = s_format_split[0]
                        ad = s_format_split[1].split(",")
                        ref_AD = ad[1]
                        alt_AD = ad[0]
                        dp = int(s_format_split[2])
                        remaining_format = ":".join(s_format_split[3:])
                        if dp == 0:
                            df.loc[index, sample] = "0/0:0,0:0:0:0,0,0"
                        elif len(ad) >2 and dp > 0:
                            df.loc[index, sample] = "0/0" + ":" + str(alt_AD) + ",0:" + str(dp) + ":" + remaining_format
                        elif len(ad) == 2 and dp > 0:
                            print(sample, df.loc[index,'POS'],df.loc[index, sample])
                    elif len(s_format_split) <= 2:
                        df.loc[index, sample] = "0/0:0,0:0:0:0,0,0"
                #df.loc[index,sample] coordinates to replace
                #all_samples_tab[sample] Dictionary with tab dataframes with all samples and variants
                #df.iloc[index,0] First columnn = POSITION to check with function and return new 0 or 1
                #post_snp = int(variant_is_present(all_samples_tab[sample], df.iloc[index,0]))
                #df.loc[index,sample] = post_snp
                #df.loc[index,sample] = variant_is_present(all_samples_tab[sample], df.iloc[index,0])
                #Substitute previous count (N) and list of samples
                #if previous_snp == 0 and post_snp == 1:
                    #Reassign number of samples (colimn with index 1)
                    #df.iloc[index,1] = df.iloc[index,1] + 1
                    #Reassign list of samples (colimn with index 2)
                    #df.iloc[index,2] = df.iloc[index,2] + "," + sample
                    if len(ad) == 2 and dp > 0:
                        pos = df.loc[index,'POS']
                        alt = df.loc[index,'ALT']
                        #print(sample,pos, alt, gt, ad, dp, remaining_format)
                        #non_genotyped any(t < 0 for t in x):
                        #print(df.iloc[index,9:].tolist())
    return df
    

In [185]:
df_test = import_VCF42_cohort_pandas(file)

In [186]:
df = repare_nongenotyped(file)

PA117-18C 21819 ./.:1,0:1:.:0,0,0
PA-004 132804 ./.:10,0:10:.:.:.:0,0,0
T-18-008 335642 ./.:1,0:1:.:0,0,0
PA-212 336325 ./.:1,0:1:.:.:.:0,0,0
P120 672330 ./.:3,0:3:.:.:.:0,0,0
P120 672332 ./.:3,0:3:.:.:.:0,0,0
PA197-18C 673564 ./.:1,0:1:.:0,0,0
T-18-008 673564 ./.:5,0:5:.:0,0,0
PA-004 839216 ./.:3,0:3:.:.:.:0,0,0
PA184-18C 839581 ./.:35,0:35:.:.:.:0,0,0
PA202-18C 839581 ./.:33,0:33:.:.:.:0,0,0
P107 839584 ./.:22,0:22:.:.:.:0,0,0
PA184-18C 839584 ./.:33,0:33:.:.:.:0,0,0
P107 839586 ./.:22,0:22:.:.:.:0,0,0
PA184-18C 839586 ./.:33,0:33:.:.:.:0,0,0
PA184-18C 1000794 ./.:59,0:59:.:.:.:0,0,0
T18-040 1093850 ./.:2,0:2:.:0,0,0
PA197-18C 1093954 ./.:1,0:1:.:.:.:0,0,0
T-18-008 1094204 ./.:1,0:1:.:0,0,0
T18-040 1094204 ./.:1,0:1:.:0,0,0
P46 1095644 ./.:13,0:13:.:.:.:0,0,0
P46 1095669 ./.:17,0:17:.:.:.:0,0,0
P46 1095672 ./.:16,0:16:.:.:.:0,0,0
PA117-18C 1095990 ./.:2,0:2:.:.:.:0,0,0
PA149-18C 1095990 ./.:4,0:4:.:.:.:0,0,0
T-18-008 1096398 ./.:1,0:1:.:.:.:0,0,0
T18-003 1189567 ./.:7,0:7:.:.:.:0,0,0

In [67]:
test = ['./.:17,0:17:.:.:.:0,0,0', '0/0:4,0:4:12:.:.:0,12,140', '0/0:10,0:10:30:.:.:0,30,405', '0/0:16,0:16:48:.:.:0,48,570', '0/0:44,0:44:0:.:.:0,0,15', '0/0:3,0:3:9:.:.:0,9,82', '0|1:2,4:6:99:0|1:150890_G_A:162,0,385:150890', './.:26,0:26:.:.:.:0,0,0', '0/0:19,0:19:57:.:.:0,57,663', './.:42,0:42:.:.:.:0,0,0', './.:33,0:33:.:.:.:0,0,0', './.:15,0:15:.:.:.:0,0,0', './.:11,0:11:.:.:.:0,0,0', './.:34,0:34:.:.:.:0,0,0', './.:18,0:18:.:.:.:0,0,0', '0|1:1,13:14:99:0|1:150890_G_A:495,0,127:150890', './.:31,0:31:.:.:.:0,0,0', './.:36,0:36:.:.:.:0,0,0', './.:17,0:17:.:.:.:0,0,0', './.:20,0:20:.:.:.:0,0,0', './.:16,0:16:.:.:.:0,0,0', '0/0:5,0:5:0:.:.:0,0,101', '0/0:9,0:9:27:.:.:0,27,342', './.:16,0:16:.:.:.:0,0,0', './.:19,0:19:.:.:.:0,0,0', '0/0:10,0:10:0:.:.:0,0,316']

In [68]:
test2 = any(x.startswith("./.") for x in test)

In [70]:
test2 = [x.startswith("./.") for x in test]

In [73]:
sum(test2)

15

In [74]:
len(test2)

26

In [18]:
df.columns[9:].tolist()

['1104',
 '1209',
 'P107',
 'P118',
 'P120',
 'P46',
 'P97',
 'PA-004',
 'PA-204',
 'PA-212',
 'PA117-18C',
 'PA119-18C',
 'PA136-18C',
 'PA149-18C',
 'PA155-18C',
 'PA157-18C',
 'PA164-18C',
 'PA176-18C',
 'PA184-18C',
 'PA197-18C',
 'PA202-18C',
 'T-18-008',
 'T18-003',
 'T18-027',
 'T18-040',
 'T18-055']

In [ ]:
d.apply(lambda x: x.str.contains('|'.join(v)))

In [ ]:
d.applymap(lambda x: any([x.startswith(v) for v in vals]))
